In [ ]:
import os

# Set the paths for your train, validation, and test datasets
base_dir = '/kaggle/input/data-255-class-competition-road-sign-detection/Road_Sign_Detection  3'

# List contents of the base directory
print("Contents of the base directory:")
print(os.listdir(base_dir))

# List contents of train, valid, and test directories
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

print("\nContents of the train directory:")
print(os.listdir(train_dir))

print("\nContents of the valid directory:")
print(os.listdir(valid_dir))

print("\nContents of the test directory:")
print(os.listdir(test_dir))


In [ ]:
import os
import random

# Set the path for your train dataset (note the double space before '3')
train_dir = '/kaggle/input/data-255-class-competition-road-sign-detection/Road_Sign_Detection  3/train'

# List the classes in the train directory
classes = os.listdir(train_dir)
print("Classes available in train directory:", classes)

# Randomly select a class to list a few images from
selected_class = random.choice(classes)
selected_class_path = os.path.join(train_dir, selected_class)

# List images in the selected class directory
images = os.listdir(selected_class_path)

# Select a few images to list (up to 5 images)
sample_images = random.sample(images, min(5, len(images)))

print("\nSample images from the selected class:")
for image in sample_images:
    print(image)


In [ ]:
import os
import random
from PIL import Image
import matplotlib.pyplot as plt

# Set the path for your train dataset (note the double space before '3')
train_dir = '/kaggle/input/data-255-class-competition-road-sign-detection/Road_Sign_Detection  3/train'

# Set the path to the images folder inside the train directory
images_dir = os.path.join(train_dir, 'images')

# List all images in the train images directory
images = os.listdir(images_dir)

# Select a few images to display (up to 5 images)
sample_images = random.sample(images, min(5, len(images)))

# Display the selected images
plt.figure(figsize=(15, 5))
for i, image_name in enumerate(sample_images):
    image_path = os.path.join(images_dir, image_name)
    image = Image.open(image_path)

    plt.subplot(1, len(sample_images), i + 1)
    plt.imshow(image)
    plt.title(f"Image: {image_name}")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import os

# Set the base path for your dataset
base_dir = '/kaggle/input/data-255-class-competition-road-sign-detection/Road_Sign_Detection  3'

# Set paths for train, validation, and test datasets
train_images_dir = os.path.join(base_dir, 'train/images')
train_labels_dir = os.path.join(base_dir, 'train/labels')
valid_images_dir = os.path.join(base_dir, 'valid/images')
valid_labels_dir = os.path.join(base_dir, 'valid/labels')
test_dir = os.path.join(base_dir, 'test')

# Training images count
train_images_count = len(os.listdir(train_images_dir))
print(f"Training images count: {train_images_count}")

# Training labels count (assuming labels are in XML files)
train_labels_count = len(os.listdir(train_labels_dir))
print(f"Training labels XML count: {train_labels_count}")

# Test files count
test_files_count = len(os.listdir(test_dir))
print(f"Test number of files count: {test_files_count}")

# Validation dataset images count
valid_images_count = len(os.listdir(valid_images_dir))
print(f"Validation dataset images count: {valid_images_count}")

# Validation labels count (assuming labels are in XML or any format)
valid_labels_count = len(os.listdir(valid_labels_dir))
print(f"Validation labels count: {valid_labels_count}")


In [ ]:
import os
import random
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image


In [ ]:
# Set the base path for your dataset
base_dir = '/kaggle/input/data-255-class-competition-road-sign-detection/Road_Sign_Detection  3'

# Paths for train, validation, and test datasets
train_images_dir = os.path.join(base_dir, 'train/images')
train_labels_dir = os.path.join(base_dir, 'train/labels')
valid_images_dir = os.path.join(base_dir, 'valid/images')
valid_labels_dir = os.path.join(base_dir, 'valid/labels')
test_dir = os.path.join(base_dir, 'test')


In [ ]:
# Define data transformations

# Training transformations with data augmentation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),           # Randomly crop the image to 224x224
    transforms.RandomHorizontalFlip(),           # Randomly flip the image horizontally
    transforms.RandomRotation(15),               # Randomly rotate the image by 15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random adjustments
    transforms.ToTensor(),                       # Convert the image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])    # Normalize using ImageNet stats
])



In [ ]:
# Validation and Test transformations - no augmentation
valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),               # Resize the image to 224x224
    transforms.ToTensor(),                       # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])    # Normalize
])


In [ ]:
class RoadSignDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = os.listdir(images_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load image
        img_name = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        # Load corresponding label (assuming YOLO format: class x_center y_center width height)
        label_name = img_name.replace('.jpg', '.txt')  # Replace with appropriate extension
        label_path = os.path.join(self.labels_dir, label_name)

        with open(label_path, 'r') as label_file:
            label_data = label_file.readlines()
            labels = []
            for line in label_data:
                components = line.strip().split()
                class_idx = int(components[0])  # Assuming first value is the class index
                labels.append(class_idx)

        # Apply transformations to the image
        if self.transform:
            image = self.transform(image)

        return image, labels



In [ ]:
def custom_collate_fn(batch):
    images = []
    labels = []

    for img, label in batch:
        images.append(img)
        labels.append(label)

    # Stack all images as tensors
    images = torch.stack(images, dim=0)

    # Labels remain as a list of variable length items
    return images, labels


In [ ]:
# Create datasets
train_dataset = RoadSignDataset(images_dir=train_images_dir, labels_dir=train_labels_dir, transform=train_transform)
valid_dataset = RoadSignDataset(images_dir=valid_images_dir, labels_dir=valid_labels_dir, transform=valid_transform)


In [ ]:
# Create DataLoader for training and validation with custom collate function
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

In [ ]:
### Visualize Augmented Images

# Function to display images
def show_images(images, labels):
    plt.figure(figsize=(15, 8))
    for i in range(len(images)):
        plt.subplot(1, len(images), i + 1)
        image = images[i].permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C)
        image = image * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])  # Unnormalize
        image = image.numpy()
        plt.imshow(image)
        plt.title(f"Labels: {labels[i]}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Get a batch of images from the training loader
data_iter = iter(train_loader)
images, labels = next(data_iter)

# Display a few training images after augmentation
show_images(images[:5], labels[:5])


In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image


In [ ]:
# Set the base path for your dataset
base_dir = '/kaggle/input/data-255-class-competition-road-sign-detection/Road_Sign_Detection  3'

# Define paths for train and validation datasets
train_images_dir = os.path.join(base_dir, 'train/images')
valid_images_dir = os.path.join(base_dir, 'valid/images')


In [ ]:
def plot_random_images(directory, num_images=5, title=""):
    """
    Plots a specified number of random images from a given directory.
    
    Args:
    - directory (str): The path to the directory containing images.
    - num_images (int): Number of images to display.
    - title (str): Title of the plot.
    """
    # List all the image files in the directory
    image_files = os.listdir(directory)
    
    # Select random images from the directory
    sample_images = random.sample(image_files, min(num_images, len(image_files)))
    
    # Plot the images
    plt.figure(figsize=(15, 5))
    plt.suptitle(title, fontsize=16)

    for i, image_name in enumerate(sample_images):
        # Load the image
        img_path = os.path.join(directory, image_name)
        image = Image.open(img_path)

        # Plot the image
        plt.subplot(1, len(sample_images), i + 1)
        plt.imshow(image)
        plt.title(image_name)
        plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# Plot random images from the training set
plot_random_images(train_images_dir, num_images=5, title="Training Set - Random Images")

# Plot random images from the validation set
plot_random_images(valid_images_dir, num_images=5, title="Validation Set - Random Images")


In [ ]:
!pip install ultralytics

In [ ]:
# Install YOLOv8 and Albumentations
!pip install ultralytics==8.0.138

# Alternatively, downgrade if the issue persists
!pip install albumentations==1.3.0

# Import necessary modules
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO
from IPython.display import Image, display
import glob
import os


In [ ]:
!pip uninstall -y ray

# Upgrade or downgrade ray to a specific version
!pip install ray==2.0.0

!pip uninstall -y ray

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data='/kaggle/input/data-255-class-competition-road-sign-detection/Road_Sign_Detection  3/data.yaml' epochs=100 imgsz=640

In [ ]:
!yolo task=detect mode=val model=/kaggle/working/runs/detect/train3/weights/best.pt data='/kaggle/input/data-255-class-competition-road-sign-detection/Road_Sign_Detection  3/data.yaml'

In [ ]:
!yolo task=detect mode=predict model=/kaggle/working/runs/detect/train3/weights/best.pt conf=0.25 source='/kaggle/input/data-255-class-competition-road-sign-detection/Road_Sign_Detection  3/test'

In [ ]:
import glob
from IPython.display import Image, display

In [ ]:
for image_path in glob.glob('runs/detect/predict4/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

In [ ]:
from ultralytics import YOLO
import csv
import os
import yaml
import pandas as pd

# Load the model
model = YOLO('/kaggle/working/runs/detect/train3/weights/best.pt')

# Path to test images
test_path = '/kaggle/input/data-255-class-competition-road-sign-detection/Road_Sign_Detection  3/test'

# Path to data.yaml file
data_yaml_path = '/kaggle/input/data-255-class-competition-road-sign-detection/Road_Sign_Detection  3/data.yaml'

# Load class names from data.yaml
with open(data_yaml_path, 'r') as file:
    data_yaml = yaml.safe_load(file)
    class_names = data_yaml['names']

# Create a CSV file to store results
with open('prediction_results_100.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['filename', 'class'])  # Use the required column names

    # Iterate through test images
    for img_path in sorted(os.listdir(test_path)):  # Sort to ensure consistent order
        if img_path.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')):
            full_img_path = os.path.join(test_path, img_path)
            results = model(full_img_path)

            best_confidence = -1
            best_class = None
            
            # Get the predicted classes and their confidences
            for r in results:
                boxes = r.boxes
                for box in boxes:
                    conf = box.conf.item()
                    pred_cls = int(box.cls.item())
                    
                    # Track the best prediction based on confidence
                    if conf > best_confidence:
                        best_confidence = conf
                        best_class = pred_cls
            
            # Write the best prediction for this image to CSV
            if best_class is not None:
                class_name = class_names[best_class]  # Get the class name from the index
            else:
                class_name = "No detection"  # Default class if nothing is detected
            
            csvwriter.writerow([img_path, class_name])

print("Predictions saved to prediction_results_100.csv")

# Print the first five rows of the CSV and verify row count
df = pd.read_csv('prediction_results_100.csv')
print(f"Number of rows in CSV: {len(df)}")
print(df.head())


In [ ]:
def calculate_iou(box1, box2):
    # box format: [x1, y1, x2, y2]
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection

    return intersection / union if union > 0 else 0

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('prediction_results_100.csv')  # Adjust the filename as needed

# Print the first five rows
print(df.head())